In [494]:
import pandas as pd
import numpy as np

import scipy.stats as stats

In [542]:
def nhl_correlation():
    
    nhl_df=pd.read_csv("nhl.csv")
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nhl_df['name'] = [i.split()[-1] for i in nhl_df['team']]
    nhl_df['area'] = [' '.join(i.split()[:-1]) for i in nhl_df['team']]
    nhl_df.replace('[*]', '', regex = True, inplace = True)

    mapper1 = {
        'Toronto Maple' : 'Toronto',
        'Detroit Red' : 'Detroit',
        'Columbus Blue' : 'Columbus',
        'Vegas Golden' : 'Vegas',

    }

    mapper2 = {
        'Leafs': 'Maple Leafs',
        'Wings' : 'Red Wings',
        'Jackets' : 'Blue Jackets',
        'Knights' : 'Golden Knights'

    }

    nhl_df = nhl_df[['area', 'name','GP', 'W', 'L', 'OL', 'PTS', 'PTS%', 'GF', 'GA', 'SRS', 'SOS',
           'RPt%', 'ROW', 'year', 'League', 'team']]



    nhl_df_2018 = nhl_df.where(nhl_df['year'] == 2018).dropna()
    nhl_df_2018['area'].replace(mapper1, inplace = True)
    nhl_df_2018['name'].replace(mapper2, inplace = True)

    mapper3 = {
        'Tampa Bay' : 'Tampa Bay Area',
        'Florida' : 'Miami–Fort Lauderdale',
        'Washington' : 'Washington, D.C.',
        'New Jersey' : 'New York City',
        'Carolina' : 'Raleigh',
        'New York' : 'New York City',
        'Minnesota' : 'Minneapolis–Saint Paul',
        'Colorado' : 'Denver',
        'Dallas' : 'Dallas–Fort Worth',
        'Vegas' : 'Las Vegas',
        'Anaheim' : 'Los Angeles',
        'San Jose' : 'San Francisco Bay Area',
        'Arizona' : 'Phoenix'
    }

    nhl_df_2018['area'] = nhl_df_2018['area'].replace(mapper3)

    nhl_df_2018 = nhl_df_2018.drop([0,9,18,26])

    nhl_df_2018['W'] = pd.to_numeric(nhl_df_2018['W'])
    nhl_df_2018['L'] = pd.to_numeric(nhl_df_2018['L'])
    
    wins = nhl_df_2018['W'].groupby(nhl_df_2018['area']).sum()
    loss = nhl_df_2018['L'].groupby(nhl_df_2018['area']).sum()

    wl_ratio = (wins / (wins+loss)).to_frame()
    wl_ratio = wl_ratio.reset_index()
    wl_ratio.rename(columns = {'area' : 'Metropolitan area', 0 : 'wl_ratio'}, inplace = True)
    cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'}, inplace = True)
    cities_ratio = pd.merge(cities, wl_ratio, how = 'inner', on = 'Metropolitan area' )

    cities_ratio = cities_ratio[['Metropolitan area', 'Population', 'wl_ratio']]
    cities_ratio['Population'] = pd.to_numeric(cities_ratio['Population'])
      
    
    population_by_region = list(cities_ratio['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(cities_ratio['wl_ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    

In [569]:
def nba_correlation():
    nba_df=pd.read_csv("nba.csv")
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    
    nba_df.replace('\(([^\)]+)\)', '', regex = True, inplace = True)
    nba_df.replace('[*]', '', regex = True, inplace = True)
    nba_df['name'] = [i.split()[-1] for i in nba_df['team']]
    nba_df['area'] = [' '.join(i.split()[:-1]) for i in nba_df['team']]

    nba_df = nba_df[[ 'area', 'name', 'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS', 'year', 'League','team']]

    nba_df_2018 = nba_df.where(nba_df['year'] == 2018).dropna()

    nba_df_2018['area'] = nba_df_2018.replace({'Portland Trail' : 'Portland'})
    nba_df_2018['name'] = nba_df_2018.replace({'Blazers' : 'Trail Blazers'})

    mapper3 = {
            'Tampa Bay' : 'Tampa Bay Area',
            'Florida' : 'Miami–Fort Lauderdale',
            'Washington' : 'Washington, D.C.',
            'New Jersey' : 'New York City',
            'Carolina' : 'Raleigh',
            'New York' : 'New York City',
            'Minnesota' : 'Minneapolis–Saint Paul',
            'Colorado' : 'Denver',
            'Dallas' : 'Dallas–Fort Worth',
            'Vegas' : 'Las Vegas',
            'Anaheim' : 'Los Angeles',
            'San Jose' : 'San Francisco Bay Area',
            'Arizona' : 'Phoenix',
            'Indiana' : 'Indianapolis',
            'Miami' : 'Miami–Fort Lauderdale',
            'Brooklyn' : 'New York City',
            'Golden State' : 'San Francisco Bay Area',
            'Utah' : 'Salt Lake City'

    }

    nba_df_2018['area'] = nba_df_2018['area'].replace(mapper3)

    nba_df_2018['W'] = pd.to_numeric(nba_df_2018['W'])
    nba_df_2018['L'] = pd.to_numeric(nba_df_2018['L'])

    wins = nba_df_2018['W'].groupby(nba_df_2018['area']).sum()
    loss = nba_df_2018['L'].groupby(nba_df_2018['area']).sum()
    wl_ratio = (wins / (wins+loss)).to_frame()

    wl_ratio = wl_ratio.reset_index()
    wl_ratio.rename(columns = {'area' : 'Metropolitan area', 0 : 'wl_ratio'}, inplace = True)
    cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'}, inplace = True)
    cities_ratio = pd.merge(cities, wl_ratio, how = 'inner', on = 'Metropolitan area' )

    cities_ratio = cities_ratio[['Metropolitan area', 'Population', 'wl_ratio']]
    cities_ratio['Population'] = pd.to_numeric(cities_ratio['Population'])

    population_by_region = list(cities_ratio['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(cities_ratio['wl_ratio'])
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [591]:
def mlb_correlation():
    mlb_df=pd.read_csv("mlb.csv")
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    mlb_df['name'] = [i.split()[-1] for i in mlb_df['team']]
    mlb_df['area'] = [' '.join(i.split()[:-1]) for i in mlb_df['team']]

    mlb_df = mlb_df[['area', 'name', 'W', 'L', 'W-L%', 'GB', 'year', 'League', 'team' ]]

    mlb_df_2018 = mlb_df.where(mlb_df['year'] == 2018).dropna()
    mlb_df_2018['name'].iloc[8] = mlb_df_2018['name'].iloc[8].replace('Sox','White Sox')

    mapper1 = {
        'Sox' : 'Red Sox',
        'Jays' : 'Blue Jays',
    }

    mapper2 = {
        'Boston Red' : 'Boston',
        'Toronto Blue' : 'Toronto',
        'Chicago White' : 'Chicago',
    }

    mlb_df_2018['name'] = mlb_df_2018['name'].replace(mapper1)
    mlb_df_2018['area'] = mlb_df_2018['area'].replace(mapper2)

    mapper3 = {
                'Tampa Bay' : 'Tampa Bay Area',
                'Florida' : 'Miami–Fort Lauderdale',
                'Washington' : 'Washington, D.C.',
                'New Jersey' : 'New York City',
                'Carolina' : 'Raleigh',
                'New York' : 'New York City',
                'Minnesota' : 'Minneapolis–Saint Paul',
                'Colorado' : 'Denver',
                'Dallas' : 'Dallas–Fort Worth',
                'Vegas' : 'Las Vegas',
                'Anaheim' : 'Los Angeles',
                'San Jose' : 'San Francisco Bay Area',
                'Arizona' : 'Phoenix',
                'Indiana' : 'Indianapolis',
                'Miami' : 'Miami–Fort Lauderdale',
                'Brooklyn' : 'New York City',
                'Golden State' : 'San Francisco Bay Area',
                'Utah' : 'Salt Lake City',
                'Oakland' : 'San Francisco Bay Area',
                'Texas' : 'Dallas–Fort Worth',
                'San Francisco' : 'San Francisco Bay Area'
     }

    mlb_df_2018['area'] = mlb_df_2018['area'].replace(mapper3)


    mlb_df_2018['W'] = pd.to_numeric(mlb_df_2018['W'])
    mlb_df_2018['L'] = pd.to_numeric(mlb_df_2018['L'])

    wins = mlb_df_2018['W'].groupby(mlb_df_2018['area']).sum()
    loss = mlb_df_2018['L'].groupby(mlb_df_2018['area']).sum()
    wl_ratio = (wins / (wins+loss)).to_frame()

    wl_ratio = wl_ratio.reset_index()
    wl_ratio.rename(columns = {'area' : 'Metropolitan area', 0 : 'wl_ratio'}, inplace = True)
    cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'}, inplace = True)
    cities_ratio = pd.merge(cities, wl_ratio, how = 'inner', on = 'Metropolitan area' )

    cities_ratio = cities_ratio[['Metropolitan area', 'Population', 'wl_ratio']]
    cities_ratio['Population'] = pd.to_numeric(cities_ratio['Population'])

    population_by_region = list(cities_ratio['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(cities_ratio['wl_ratio'])
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [607]:
def nfl_correlation():
    nfl_df=pd.read_csv("nfl.csv")
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nfl_df['name'] = [i.split()[-1] for i in nfl_df['team']]
    nfl_df['area'] = [' '.join(i.split()[:-1]) for i in nfl_df['team']]
    nfl_df.replace('[*]', '', regex = True, inplace = True)
    nfl_df.replace('[+]', '', regex = True, inplace = True)

    nfl_df = nfl_df[['area', 'name', 'DSRS', 'L', 'League', 'MoV', 'OSRS', 'PA', 'PD', 'PF', 'SRS', 'SoS',
           'T', 'W', 'W-L%', 'year', 'team']]

    nfl_df_2018 = nfl_df.where(nfl_df['year'] == 2018).dropna()
    nfl_df_2018 = nfl_df_2018.drop(list(range(0,39, 5)))
    nfl_df_2018 = nfl_df_2018.reset_index()
    nfl_df_2018 = nfl_df_2018.drop('index', axis = 1)

    mapper3 = {
                    'Tampa Bay' : 'Tampa Bay Area',
                    'Florida' : 'Miami–Fort Lauderdale',
                    'Washington' : 'Washington, D.C.',
                    'New Jersey' : 'New York City',
                    'Carolina' : 'Raleigh',
                    'New York' : 'New York City',
                    'Minnesota' : 'Minneapolis–Saint Paul',
                    'Colorado' : 'Denver',
                    'Dallas' : 'Dallas–Fort Worth',
                    'Vegas' : 'Las Vegas',
                    'Anaheim' : 'Los Angeles',
                    'San Jose' : 'San Francisco Bay Area',
                    'Arizona' : 'Phoenix',
                    'Indiana' : 'Indianapolis',
                    'Miami' : 'Miami–Fort Lauderdale',
                    'Brooklyn' : 'New York City',
                    'Golden State' : 'San Francisco Bay Area',
                    'Utah' : 'Salt Lake City',
                    'Oakland' : 'San Francisco Bay Area',
                    'Texas' : 'Dallas–Fort Worth',
                    'San Francisco' : 'San Francisco Bay Area',
                    'New England' : 'Boston',
                    'Tennessee' : 'Nashville'
    }

    nfl_df_2018['area'] = nfl_df_2018['area'].replace(mapper3)

    nfl_df_2018['W'] = pd.to_numeric(nfl_df_2018['W'])
    nfl_df_2018['L'] = pd.to_numeric(nfl_df_2018['L'])

    wins = nfl_df_2018['W'].groupby(nfl_df_2018['area']).sum()
    loss = nfl_df_2018['L'].groupby(nfl_df_2018['area']).sum()
    wl_ratio = (wins / (wins+loss)).to_frame()

    wl_ratio = wl_ratio.reset_index()
    wl_ratio.rename(columns = {'area' : 'Metropolitan area', 0 : 'wl_ratio'}, inplace = True)
    cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'}, inplace = True)
    cities_ratio = pd.merge(cities, wl_ratio, how = 'inner', on = 'Metropolitan area' )

    cities_ratio = cities_ratio[['Metropolitan area', 'Population', 'wl_ratio']]
    cities_ratio['Population'] = pd.to_numeric(cities_ratio['Population'])

    population_by_region = list(cities_ratio['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(cities_ratio['wl_ratio'])

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [687]:
def nhl_wl_ratio():
    nhl_df=pd.read_csv("nhl.csv")
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nhl_df['name'] = [i.split()[-1] for i in nhl_df['team']]
    nhl_df['area'] = [' '.join(i.split()[:-1]) for i in nhl_df['team']]
    nhl_df.replace('[*]', '', regex = True, inplace = True)

    mapper1 = {
        'Toronto Maple' : 'Toronto',
        'Detroit Red' : 'Detroit',
        'Columbus Blue' : 'Columbus',
        'Vegas Golden' : 'Vegas',

    }

    mapper2 = {
        'Leafs': 'Maple Leafs',
        'Wings' : 'Red Wings',
        'Jackets' : 'Blue Jackets',
        'Knights' : 'Golden Knights'

    }

    nhl_df = nhl_df[['area', 'name','GP', 'W', 'L', 'OL', 'PTS', 'PTS%', 'GF', 'GA', 'SRS', 'SOS',
           'RPt%', 'ROW', 'year', 'League', 'team']]



    nhl_df_2018 = nhl_df.where(nhl_df['year'] == 2018).dropna()
    nhl_df_2018['area'].replace(mapper1, inplace = True)
    nhl_df_2018['name'].replace(mapper2, inplace = True)

    mapper3 = {
        'Tampa Bay' : 'Tampa Bay Area',
        'Florida' : 'Miami–Fort Lauderdale',
        'Washington' : 'Washington, D.C.',
        'New Jersey' : 'New York City',
        'Carolina' : 'Raleigh',
        'New York' : 'New York City',
        'Minnesota' : 'Minneapolis–Saint Paul',
        'Colorado' : 'Denver',
        'Dallas' : 'Dallas–Fort Worth',
        'Vegas' : 'Las Vegas',
        'Anaheim' : 'Los Angeles',
        'San Jose' : 'San Francisco Bay Area',
        'Arizona' : 'Phoenix'
    }

    nhl_df_2018['area'] = nhl_df_2018['area'].replace(mapper3)

    nhl_df_2018 = nhl_df_2018.drop([0,9,18,26])

    nhl_df_2018['W'] = pd.to_numeric(nhl_df_2018['W'])
    nhl_df_2018['L'] = pd.to_numeric(nhl_df_2018['L'])
    
    wins = nhl_df_2018['W'].groupby(nhl_df_2018['area']).sum()
    loss = nhl_df_2018['L'].groupby(nhl_df_2018['area']).sum()

    wl_ratio = (wins / (wins+loss)).to_frame()
    wl_ratio = wl_ratio.reset_index()
    wl_ratio.rename(columns = {'area' : 'Metropolitan area', 0 : 'nhl_wl_ratio'}, inplace = True)
    return  wl_ratio

def nba_wl_ratio():
    nba_df=pd.read_csv("nba.csv")
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    
    nba_df.replace('\(([^\)]+)\)', '', regex = True, inplace = True)
    nba_df.replace('[*]', '', regex = True, inplace = True)
    nba_df['name'] = [i.split()[-1] for i in nba_df['team']]
    nba_df['area'] = [' '.join(i.split()[:-1]) for i in nba_df['team']]

    nba_df = nba_df[[ 'area', 'name', 'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS', 'year', 'League','team']]

    nba_df_2018 = nba_df.where(nba_df['year'] == 2018).dropna()

    nba_df_2018['area'] = nba_df_2018.replace({'Portland Trail' : 'Portland'})
    nba_df_2018['name'] = nba_df_2018.replace({'Blazers' : 'Trail Blazers'})

    mapper3 = {
            'Tampa Bay' : 'Tampa Bay Area',
            'Florida' : 'Miami–Fort Lauderdale',
            'Washington' : 'Washington, D.C.',
            'New Jersey' : 'New York City',
            'Carolina' : 'Raleigh',
            'New York' : 'New York City',
            'Minnesota' : 'Minneapolis–Saint Paul',
            'Colorado' : 'Denver',
            'Dallas' : 'Dallas–Fort Worth',
            'Vegas' : 'Las Vegas',
            'Anaheim' : 'Los Angeles',
            'San Jose' : 'San Francisco Bay Area',
            'Arizona' : 'Phoenix',
            'Indiana' : 'Indianapolis',
            'Miami' : 'Miami–Fort Lauderdale',
            'Brooklyn' : 'New York City',
            'Golden State' : 'San Francisco Bay Area',
            'Utah' : 'Salt Lake City'

    }

    nba_df_2018['area'] = nba_df_2018['area'].replace(mapper3)

    nba_df_2018['W'] = pd.to_numeric(nba_df_2018['W'])
    nba_df_2018['L'] = pd.to_numeric(nba_df_2018['L'])

    wins = nba_df_2018['W'].groupby(nba_df_2018['area']).sum()
    loss = nba_df_2018['L'].groupby(nba_df_2018['area']).sum()
    wl_ratio = (wins / (wins+loss)).to_frame()

    wl_ratio = wl_ratio.reset_index()
    wl_ratio.rename(columns = {'area' : 'Metropolitan area', 0 : 'nba_wl_ratio'}, inplace = True)
    
    return wl_ratio

def mlb_wl_ratio():
    mlb_df=pd.read_csv("mlb.csv")
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    mlb_df['name'] = [i.split()[-1] for i in mlb_df['team']]
    mlb_df['area'] = [' '.join(i.split()[:-1]) for i in mlb_df['team']]

    mlb_df = mlb_df[['area', 'name', 'W', 'L', 'W-L%', 'GB', 'year', 'League', 'team' ]]

    mlb_df_2018 = mlb_df.where(mlb_df['year'] == 2018).dropna()
    mlb_df_2018['name'].iloc[8] = mlb_df_2018['name'].iloc[8].replace('Sox','White Sox')

    mapper1 = {
        'Sox' : 'Red Sox',
        'Jays' : 'Blue Jays',
    }

    mapper2 = {
        'Boston Red' : 'Boston',
        'Toronto Blue' : 'Toronto',
        'Chicago White' : 'Chicago',
    }

    mlb_df_2018['name'] = mlb_df_2018['name'].replace(mapper1)
    mlb_df_2018['area'] = mlb_df_2018['area'].replace(mapper2)

    mapper3 = {
                'Tampa Bay' : 'Tampa Bay Area',
                'Florida' : 'Miami–Fort Lauderdale',
                'Washington' : 'Washington, D.C.',
                'New Jersey' : 'New York City',
                'Carolina' : 'Raleigh',
                'New York' : 'New York City',
                'Minnesota' : 'Minneapolis–Saint Paul',
                'Colorado' : 'Denver',
                'Dallas' : 'Dallas–Fort Worth',
                'Vegas' : 'Las Vegas',
                'Anaheim' : 'Los Angeles',
                'San Jose' : 'San Francisco Bay Area',
                'Arizona' : 'Phoenix',
                'Indiana' : 'Indianapolis',
                'Miami' : 'Miami–Fort Lauderdale',
                'Brooklyn' : 'New York City',
                'Golden State' : 'San Francisco Bay Area',
                'Utah' : 'Salt Lake City',
                'Oakland' : 'San Francisco Bay Area',
                'Texas' : 'Dallas–Fort Worth',
                'San Francisco' : 'San Francisco Bay Area'
     }

    mlb_df_2018['area'] = mlb_df_2018['area'].replace(mapper3)


    mlb_df_2018['W'] = pd.to_numeric(mlb_df_2018['W'])
    mlb_df_2018['L'] = pd.to_numeric(mlb_df_2018['L'])

    wins = mlb_df_2018['W'].groupby(mlb_df_2018['area']).sum()
    loss = mlb_df_2018['L'].groupby(mlb_df_2018['area']).sum()
    wl_ratio = (wins / (wins+loss)).to_frame()

    wl_ratio = wl_ratio.reset_index()
    wl_ratio.rename(columns = {'area' : 'Metropolitan area', 0 : 'mlb_wl_ratio'}, inplace = True)
    
    return wl_ratio

def nfl_wl_ratio():
    nfl_df=pd.read_csv("nfl.csv")
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nfl_df['name'] = [i.split()[-1] for i in nfl_df['team']]
    nfl_df['area'] = [' '.join(i.split()[:-1]) for i in nfl_df['team']]
    nfl_df.replace('[*]', '', regex = True, inplace = True)
    nfl_df.replace('[+]', '', regex = True, inplace = True)

    nfl_df = nfl_df[['area', 'name', 'DSRS', 'L', 'League', 'MoV', 'OSRS', 'PA', 'PD', 'PF', 'SRS', 'SoS',
           'T', 'W', 'W-L%', 'year', 'team']]

    nfl_df_2018 = nfl_df.where(nfl_df['year'] == 2018).dropna()
    nfl_df_2018 = nfl_df_2018.drop(list(range(0,39, 5)))
    nfl_df_2018 = nfl_df_2018.reset_index()
    nfl_df_2018 = nfl_df_2018.drop('index', axis = 1)

    mapper3 = {
                    'Tampa Bay' : 'Tampa Bay Area',
                    'Florida' : 'Miami–Fort Lauderdale',
                    'Washington' : 'Washington, D.C.',
                    'New Jersey' : 'New York City',
                    'Carolina' : 'Raleigh',
                    'New York' : 'New York City',
                    'Minnesota' : 'Minneapolis–Saint Paul',
                    'Colorado' : 'Denver',
                    'Dallas' : 'Dallas–Fort Worth',
                    'Vegas' : 'Las Vegas',
                    'Anaheim' : 'Los Angeles',
                    'San Jose' : 'San Francisco Bay Area',
                    'Arizona' : 'Phoenix',
                    'Indiana' : 'Indianapolis',
                    'Miami' : 'Miami–Fort Lauderdale',
                    'Brooklyn' : 'New York City',
                    'Golden State' : 'San Francisco Bay Area',
                    'Utah' : 'Salt Lake City',
                    'Oakland' : 'San Francisco Bay Area',
                    'Texas' : 'Dallas–Fort Worth',
                    'San Francisco' : 'San Francisco Bay Area',
                    'New England' : 'Boston',
                    'Tennessee' : 'Nashville'
    }

    nfl_df_2018['area'] = nfl_df_2018['area'].replace(mapper3)

    nfl_df_2018['W'] = pd.to_numeric(nfl_df_2018['W'])
    nfl_df_2018['L'] = pd.to_numeric(nfl_df_2018['L'])

    wins = nfl_df_2018['W'].groupby(nfl_df_2018['area']).sum()
    loss = nfl_df_2018['L'].groupby(nfl_df_2018['area']).sum()
    wl_ratio = (wins / (wins+loss)).to_frame()

    wl_ratio = wl_ratio.reset_index()
    wl_ratio.rename(columns = {'area' : 'Metropolitan area', 0 : 'nfl_wl_ratio'}, inplace = True)
    
    return wl_ratio

def sports_team_performance():
    nfl_wl = nfl_wl_ratio()
    nba_wl = nba_wl_ratio()
    nhl_wl = nhl_wl_ratio()
    mlb_wl = mlb_wl_ratio()


    NFL_ratios = []
    merged = pd.merge(nfl_wl, nfl_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nfl_wl_ratio_x'], merged['nfl_wl_ratio_y'] )[1]
    NFL_ratios.append(p_value)
    merged = pd.merge(nfl_wl, nba_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nfl_wl_ratio'], merged['nba_wl_ratio'] )[1]
    NFL_ratios.append(p_value)
    merged = pd.merge(nfl_wl, nhl_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nfl_wl_ratio'], merged['nhl_wl_ratio'] )[1]
    NFL_ratios.append(p_value)
    merged = pd.merge(nfl_wl, mlb_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nfl_wl_ratio'], merged['mlb_wl_ratio'] )[1]
    NFL_ratios.append(p_value)

    NBA_ratios = []
    merged = pd.merge(nba_wl, nfl_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nba_wl_ratio'], merged['nfl_wl_ratio'] )[1]
    NBA_ratios.append(p_value)
    merged = pd.merge(nba_wl, nba_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nba_wl_ratio_x'], merged['nba_wl_ratio_y'] )[1]
    NBA_ratios.append(p_value)
    merged = pd.merge(nba_wl, nhl_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nba_wl_ratio'], merged['nhl_wl_ratio'] )[1]
    NBA_ratios.append(p_value)
    merged = pd.merge(nba_wl, mlb_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nba_wl_ratio'], merged['mlb_wl_ratio'] )[1]
    NBA_ratios.append(p_value)

    NHL_ratios = []
    merged = pd.merge(nhl_wl, nfl_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nhl_wl_ratio'], merged['nfl_wl_ratio'] )[1]
    NHL_ratios.append(p_value)
    merged = pd.merge(nhl_wl, nba_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nhl_wl_ratio'], merged['nba_wl_ratio'] )[1]
    NHL_ratios.append(p_value)
    merged = pd.merge(nhl_wl, nhl_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nhl_wl_ratio_x'], merged['nhl_wl_ratio_y'] )[1]
    NHL_ratios.append(p_value)
    merged = pd.merge(nhl_wl, mlb_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['nhl_wl_ratio'], merged['mlb_wl_ratio'] )[1]
    NHL_ratios.append(p_value)


    MLB_ratios = []
    merged = pd.merge(mlb_wl, nfl_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['mlb_wl_ratio'], merged['nfl_wl_ratio'] )[1]
    MLB_ratios.append(p_value)
    merged = pd.merge(mlb_wl, nba_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['mlb_wl_ratio'], merged['nba_wl_ratio'] )[1]
    MLB_ratios.append(p_value)
    merged = pd.merge(mlb_wl, nhl_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['mlb_wl_ratio'], merged['nhl_wl_ratio'] )[1]
    MLB_ratios.append(p_value)
    merged = pd.merge(mlb_wl, mlb_wl, 'inner', 'Metropolitan area')
    p_value = stats.ttest_rel(merged['mlb_wl_ratio_x'], merged['mlb_wl_ratio_y'] )[1]
    MLB_ratios.append(p_value)


    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    p_values['NFL'] = NFL_ratios
    p_values['NBA'] = NBA_ratios
    p_values['NHL'] = NHL_ratios
    p_values['MLB'] = MLB_ratios

    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    
    return p_values

In [688]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.940611,0.024777,0.802384
NBA,0.940611,NaN,0.022316,0.951046
NHL,0.024777,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN
